# COBRA (part2)

In [ ]:
import cobra
from cobra.test import create_test_model
from cobra.flux_analysis import pfba
from cobra.flux_analysis import flux_variability_analysis as fva

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model = create_test_model("textbook")
iJO1366 = create_test_model("ecoli")

## loopless FVA

In [ ]:
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
fva(model, reaction_list=loop_reactions, loopless=False)

In [ ]:
fva(model, reaction_list=loop_reactions, loopless=True)

In [ ]:
# the pfba result is loopless
pfba(model, reactions=loop_reactions)

## sampling

In [ ]:
from cobra.sampling.sampling import sample

sample_sol = sample(model, n=500, method="achr", seed=42)
sample_sol

In [ ]:
sns.displot(sample_sol["ACALD"], kde=True)

In [ ]:
sns.set(style="white")
f, ax = plt.subplots(figsize=(22, 18))
sns.heatmap(sample_sol.loc[:, (sample_sol != 0).all(axis=0)].corr(), cmap="YlGnBu", center=0, square=True, linewidths=.1, cbar_kws={"shrink": .5})

## Gene and reaction essentiality

In [ ]:

from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

print('complete model: ', model.slim_optimize())
with model:
    model.reactions.PFK.knock_out()
    print('pfk knocked out: ', model.slim_optimize())

In [ ]:
gene_deletion_results = single_gene_deletion(model)

gene_deletion_results

In [ ]:
# essential genes
gene_deletion_results[(gene_deletion_results["status"] != "optimal") | gene_deletion_results["growth"] < 0.1]

In [ ]:
rxn_deletion_results = single_reaction_deletion(model)
rxn_deletion_results

In [ ]:
# essential reactions
rxn_deletion_results[(rxn_deletion_results["status"] != "optimal") | rxn_deletion_results["growth"] < 0.1]

In [ ]:
del_double_genes = double_gene_deletion(model)
del_double_genes[(del_double_genes["status"] != "optimal") | del_double_genes["growth"] < 0.1]

In [ ]:
del_double_rxns = double_reaction_deletion(model)
del_double_rxns[(del_double_rxns["status"] != "optimal") | del_double_rxns["growth"] < 0.1]

## Consistency testing

In [ ]:
# find block reactions (cannot generate flux)
blocked_reactions = cobra.flux_analysis.find_blocked_reactions(iJO1366)
blocked_reactions

In [ ]:
from cobra.flux_analysis import fastcc

consistent_iJO1366 = fastcc(iJO1366)
consistent_iJO1366

In [ ]:
iJO1366

In [ ]:
len(set([r.id for r in consistent_iJO1366.reactions]) ^ (set([r.id for r in iJO1366.reactions]) - set(blocked_reactions)))

In [ ]:
set([r.id for r in consistent_iJO1366.reactions]) ^ (set([r.id for r in iJO1366.reactions]) - set(blocked_reactions))